In [ ]:
#!pip install scikit-learn
#!pip install graphviz

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import wilcoxon
from scipy import stats
from sklearn.model_selection import train_test_split

<h1>EDA</h1>

In [ ]:
df=pd.read_csv('churn.csv')
df = df.rename(columns=str.lower)

original_df = df
df.info()

In [ ]:
print('Number of rows:',df.shape[0]) 
print('Number of columns:',df.shape[1]) 

In [ ]:
df.isnull().sum()

In [ ]:
print(f"The amount of duplicated numbers are {df.duplicated().sum()}")

In [ ]:
df.head(20)

In [ ]:
# הפכיה לערכים מספריים על מנת להציג את הנתונים
df['totalcharges'] = df['totalcharges'].str.strip() 
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df.isnull().sum()


In [ ]:
# הצגת השורות הבעייתיות
problem_rows = df[df['totalcharges'].isna()]

problem_rows = problem_rows[['customerid', 'totalcharges']]

print(problem_rows['customerid'].count(),problem_rows)

In [ ]:
df = df.dropna(subset=['totalcharges'])

In [ ]:
df.isnull().sum()

In [ ]:
df['churn'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'lightcoral'])
plt.title('churn distribution')
plt.ylabel('') 
plt.show()


In [ ]:
#3. התפלגות משתנה עזיבה  (לפני חלוקה):
sns.countplot(x='churn', data=df, palette='Set2')
plt.title('churn Distribution')
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
df[['tenure']].describe()

In [ ]:

#df[df['churn']==0]['tenure'].describe()

In [ ]:
#פירוט 1
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# גרף 1 התפלגות totalcharges
axes[0].hist(df['totalcharges'], bins=20, color='skyblue', edgecolor='black')
axes[0].set_title('Distribution of totalcharges')
axes[0].set_xlabel('totalcharges')
axes[0].set_ylabel('Frequency')

# גרף 2 התפלגות Tenure
axes[1].hist(df['tenure'], bins=20, color='skyblue', edgecolor='black')
axes[1].set_title('Distribution of Tenure')
axes[1].set_xlabel('Tenure')
axes[1].set_ylabel('Frequency')

plt.show()

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(12, 12))

# פילוח לפי מגדר
gender_counts = df['gender'].value_counts()
axes[0, 0].pie(gender_counts, labels=["Male", "Female"], autopct='%1.1f%%', colors=['skyblue', 'lightcoral'])
axes[0, 0].set_title('Segmentation by gender')

def get_churned_percentage(group_name, group_value):
    churned_group_a_count = df[(df[group_name]==group_value) & (df['churn']=='Yes')].shape[0]
    total_group_a_count = df[df[group_name]==group_value].shape[0]
    return (churned_group_a_count/total_group_a_count)

group_a_percentage = get_churned_percentage('gender', 'Male')
group_b_percentage = get_churned_percentage('gender', 'Female')
axes[0, 1].pie([group_a_percentage,group_b_percentage], labels=["churned_male", "churned_female"], autopct='%1.1f%%', colors=['skyblue', 'lightcoral'])
axes[0, 1].set_title('Segmentation by gender')

# פילוח לפי בן/בת זוג
partner_counts = df['partner'].value_counts()
axes[1, 0].pie(partner_counts, labels=['Without a spouse', 'With a spouse'], autopct='%1.1f%%', colors=['lightgreen', 'lightpink'])
axes[1, 0].set_title('Segmentation by spouse')

group_a_percentage = get_churned_percentage('partner', 'Yes')
group_b_percentage = get_churned_percentage('partner', 'No')
axes[1, 1].pie([group_a_percentage,group_b_percentage], labels=["churned_with_spouse", "churned_without_spouse"], autopct='%1.1f%%', colors=['lightgreen', 'lightpink'])
axes[1, 1].set_title('Segmentation by partner')

# פילוח לפי תלויים
dependents_counts = df['dependents'].value_counts()
axes[2, 0].pie(dependents_counts, labels=['Without dependents', 'With dependents'], autopct='%1.1f%%', colors=['lightgrey', 'lightyellow'])
axes[2, 0].set_title('Segmentation by dependents')

group_a_percentage = get_churned_percentage('dependents', 'Yes')
group_b_percentage = get_churned_percentage('dependents', 'No')
axes[2, 1].pie([group_a_percentage,group_b_percentage], labels=["churned_with_dependents", "churned_without_dependents"], autopct='%1.1f%%', colors=['lightgrey', 'lightyellow'])
axes[2, 1].set_title('Segmentation by dependents')

# # פילוח לפי עזיבה
# churn_counts = df['churn'].value_counts()
# axes[1, 1].pie(churn_counts, labels=['Didn\'t leave', 'Leave'], autopct='%1.1f%%', colors=['lightblue', 'salmon'])
# axes[1, 1].set_title('Segmentation by departure')


plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# גרף 1 Boxplot ל-totalcharges
df.boxplot(column='totalcharges', ax=axes[0])
axes[0].set_title('Boxplot of Total Charges')
axes[0].set_ylabel('Total Charges')

# גרף 2 Boxplot ל-Tenure
df.boxplot(column='tenure', ax=axes[1])
axes[1].set_title('Boxplot of Tenure')
axes[1].set_ylabel('Tenure')

plt.show()

In [ ]:
#  גרף התפלגות עמודה מספרית (monthlycharges)
plt.figure(figsize=(12, 6))
sns.histplot(df['monthlycharges'], bins=30, kde=True)
plt.title('Monthly distribution')
plt.xlabel('Monthly Charges')
plt.grid(axis='y', linestyle='--')
plt.show()

#  גרף פילוח לפי contract ו-churn
plt.figure(figsize=(12, 6))
sns.countplot(x='contract',  data=df) 
plt.title('contract Type vs. churn')
plt.grid(axis='y', linestyle='--')
plt.show()

<h1>Data Engineering / Prep </h1>

In [ ]:
def create_dummies(data_frame, column_name):
    return pd.get_dummies(data_frame, columns=[column_name], prefix=column_name, drop_first=True,dtype=int)

In [ ]:
df.loc[df.churn == 'No','churn'] =0
df.loc[df.churn == 'Yes','churn'] =1
df['churn'] = df['churn'].astype(int)

df.loc[df.gender == 'Male','gender']=0
df.loc[df.gender == 'Female','gender']=1
df['gender'] = df['gender'].astype(int)

df.loc[df.partner == 'No','partner']=0
df.loc[df.partner == 'Yes','partner']=1
df['partner'] = df['partner'].astype(int)

df.loc[df.dependents == 'No','dependents']=0
df.loc[df.dependents == 'Yes','dependents']=1
df['dependents'] = df['dependents'].astype(int)

df.loc[df.phoneservice == 'No','phoneservice'] =0
df.loc[df.phoneservice == 'Yes','phoneservice'] =1
df['phoneservice'] = df['phoneservice'].astype(int)

df.loc[df.paperlessbilling == 'No','paperlessbilling'] = 0
df.loc[df.paperlessbilling == 'Yes','paperlessbilling'] =1
df['paperlessbilling'] = df['paperlessbilling'].astype(int)

df = create_dummies(df, 'contract')
df = create_dummies(df, 'multiplelines')
df = create_dummies(df, 'internetservice')
df = create_dummies(df, 'onlinesecurity')
df = create_dummies(df, 'onlinebackup')
df = create_dummies(df, 'deviceprotection')
df = create_dummies(df, 'techsupport')
df = create_dummies(df, 'streamingtv')
df = create_dummies(df, 'streamingmovies')
df = create_dummies(df, 'paymentmethod')

df = df.drop('customerid', axis=1)

print(df.info())
df.head(20)

In [ ]:
df['combined'] = df['internetservice_Fiber optic'] | df['internetservice_No'] | \
       df['onlinesecurity_No internet service'] | df['onlinesecurity_Yes'] | \
       df['onlinebackup_No internet service'] | df['onlinebackup_Yes'] | \
       df['deviceprotection_No internet service'] | df['deviceprotection_Yes'] | \
       df['techsupport_No internet service'] | df['techsupport_Yes'] | \
       df['streamingtv_No internet service'] | df['streamingtv_Yes'] | \
       df['streamingmovies_No internet service'] | df['streamingmovies_Yes']

df.drop(['internetservice_Fiber optic', 'internetservice_No','onlinesecurity_No internet service','onlinesecurity_Yes'], axis=1, inplace=True)
df.drop(['onlinebackup_No internet service','onlinebackup_Yes','deviceprotection_No internet service', 'deviceprotection_Yes'], axis=1, inplace=True)
df.drop(['techsupport_No internet service', 'techsupport_Yes','streamingtv_No internet service', 'streamingtv_Yes'], axis=1, inplace=True)
df.drop(['streamingmovies_No internet service', 'streamingmovies_Yes'], axis=1, inplace=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
#פירוט 3

# סט 1 חלוקה רחבה
def categorize_tenure_set1(tenure):
    if tenure <= 12:
        return '0-12'
    elif 13 <= tenure <= 36:
        return '13-36'
    elif 37 <= tenure <= 60:
        return '37-60'
    else:
        return '60+'

# סט 2 (6 חודשים)
def categorize_tenure_set2(tenure):
    if tenure <= 6:
        return '0-6'
    elif 7 <= tenure <= 12:
        return '7-12'
    elif 13 <= tenure <= 18:
        return '13-18'
    elif 19 <= tenure <= 24:
        return '19-24'
    elif 25 <= tenure <= 30:
        return '25-30'
    elif 31 <= tenure <= 36:
        return '31-36'
    elif 37 <= tenure <= 42:
        return '37-42'
    elif 43 <= tenure <= 48:
        return '43-48'
    elif 49 <= tenure <= 54:
        return '49-54'
    elif 55 <= tenure <= 60:
        return '55-60'
    else:
        return '60+'

# סט 3 (10 חודשים)
def categorize_tenure_set3(tenure):
    if tenure <= 10:
        return '0-10'
    elif 11 <= tenure <= 20:
        return '11-20'
    elif 21 <= tenure <= 30:
        return '21-30'
    elif 31 <= tenure <= 40:
        return '31-40'
    elif 41 <= tenure <= 50:
        return '41-50'
    elif 51 <= tenure <= 60:
        return '51-60'
    else:
        return '60+'

df['tenure_category_set1'] = df['tenure'].apply(categorize_tenure_set1)
df['tenure_category_set2'] = df['tenure'].apply(categorize_tenure_set2)
df['tenure_category_set3'] = df['tenure'].apply(categorize_tenure_set3)

categories_set1 = ['0-12', '13-36', '37-60', '60+']
categories_set2 = ['0-6', '7-12', '13-18', '19-24', '25-30', '31-36', '37-42', '43-48', '49-54', '55-60', '60+']
categories_set3 = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '60+']

In [ ]:
stats_set1 = df.groupby(['churn','tenure_category_set1'])['monthlycharges'].agg(['mean'])
stats_set2 = df.groupby(['churn','tenure_category_set2'])['monthlycharges'].agg(['mean'])
stats_set3 = df.groupby(['churn','tenure_category_set3'])['monthlycharges'].agg(['mean'])

avg_std_set1 = stats_set1.groupby('churn')[['mean']].agg('mean')
avg_std_set2 = stats_set2.groupby('churn')[['mean']].agg('mean')
avg_std_set3 = stats_set3.groupby('churn')[['mean']].agg('mean')

In [ ]:
print(stats_set1)
print(avg_std_set1)
print(stats_set2)
print(avg_std_set2)
print(stats_set3)
print(avg_std_set3)

In [ ]:
unstacked = stats_set1.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))

#plt.plot(x='tenure_category_set1', hue='churn', data=stats_set1, palette='coolwarm')
plt.title('churn segmentation by avg monthlycharges (Set 1)')
plt.xlabel('tenure groups')
plt.legend(title='churn')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
unstacked = stats_set2.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))

plt.title('churn segmentation by avg monthlycharges (Set 2)')
plt.xlabel('tenure groups')
plt.legend(title='churn')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
unstacked = stats_set3.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))

plt.title('churn segmentation by avg monthlycharges (Set 3)')
plt.xlabel('tenure groups')
plt.legend(title='churn')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
data_set2 = df.groupby(['churn','tenure_category_set2'])['partner'].agg(['count'])
unstacked = data_set2.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))
plt.title('churn segmentation by amount of people in tenure group (Set 2)')
plt.xlabel('tenure groups')
plt.legend(title='tenure')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
data_set1 = df.groupby(['churn','tenure_category_set1'])['partner'].agg(['count'])
unstacked = data_set1.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))
plt.title('churn segmentation by amount of people in tenure group (Set 1)')
plt.xlabel('tenure groups')
plt.legend(title='tenure')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
data_set3 = df.groupby(['churn','tenure_category_set3'])['partner'].agg(['count'])
unstacked = data_set3.unstack()
unstacked.plot(kind='bar', figsize=(12, 6))
plt.title('churn segmentation by amount of people in tenure group (Set 3)')
plt.xlabel('tenure groups')
plt.legend(title='tenure')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# חישוב קורלציות אחרי המרה
corr_matrix = df.corr(numeric_only=True)

# יצירת heatmap לכלל הנתונים
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix Heatmap")
plt.show()
#corr_matrix.index

In [ ]:
corr_totalcharges = corr_matrix[['totalcharges']].sort_values(by='totalcharges', ascending=False)


plt.figure(figsize=(6, 8))
sns.heatmap(corr_totalcharges, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation with Total Charges")
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x=df['tenure'], y=df['totalcharges'], alpha=0.5)
plt.title("Total Charges vs Tenure")
plt.xlabel("Tenure")
plt.ylabel("Total Charges")
plt.grid()
plt.show()


#     נחוץ??
plt.figure(figsize=(8, 5))
sns.scatterplot(x=df['monthlycharges'], y=df['totalcharges'], alpha=0.5)
plt.title("Total Charges vs Monthly Charges")
plt.xlabel("Monthly Charges")
plt.ylabel("Total Charges")
plt.grid()
plt.show()


In [ ]:
#ML
df = df.drop(columns=['tenure_category_set1'])
df = df.drop(columns=['tenure_category_set2'])
df = df.drop(columns=['tenure_category_set3'])
